# 09 Calculate F1 vs. F2 Differentials and Addl Processing
- Calculate the F1 and F2 differentials
- Add other feature engineering here
- Train test split here

## Imports

In [36]:
import pandas as pd
import pickle

In [23]:
df = pd.read_csv('../../02_Data/02_Processed_Data/train.csv', index_col=0)

In [24]:
f1_columns = [col for col in df.columns[9:] if col[:2]=='f1']
f2_columns = [col for col in df.columns[9:] if col[:2]=='f2']

In [25]:
def Perform_calc_fight_stat_differential(df):
    f1_columns = [col for col in df.columns[9:] if col[:2]=='f1']
    for_calc_columns = [col[3:] for col in f1_columns] # Remove the F1_ part
    num_columns_before = df.shape[1]
    columns_before = df.columns
    print(f'num columns before: {num_columns_before}')
    for key, col in enumerate(for_calc_columns):
        df = calc_fight_stat_differential(df, col)
        if key%100==0:
            print(key)
    num_columns_after = df.shape[1]
    columns_after = df.columns
    print(f"# Columns Added: {num_columns_after-num_columns_before}")
    print(f'num columns after: {num_columns_after}')
    new_columns = list(set(columns_after) - set(columns_before))
    return df, new_columns

In [26]:
def calc_fight_stat_differential(df, col_root):
    f1 = 'f1_' + col_root
    f2 = 'f2_' + col_root
    diff = col_root + '_diff'
    df[diff] = df.apply(lambda x: (x[f1] - x[f2])/(x[f1] + x[f2]) if (x[f1] + x[f2]) != 0 else 0, axis=1)
    return df

# calc_fight_stat_differential(df, col_root)

In [27]:
df, new_diff = Perform_calc_fight_stat_differential(df)

num columns before: 454
0
100
200
# Columns Added: 222
num columns after: 676


In [28]:
df.head(2)

eventid  fightid     f1_fullname       f2_fullname  f1_fighterid  \
0      644     4531  Anthony Perosh        Ryan Bader           175   
1      701     5191  Brandon Thatch  Benson Henderson          1331   

   f2_fighterid f1_outcome        f1_slug          f2_slug  f1_height  \
0           503       Loss  anthonyperosh        ryanbader       75.0   
1          1066       Loss  brandonthatch  bensonhenderson       74.0   

                     ...                      \
0                    ...                       
1                    ...                       

   total_strikes_percent_diff_avg_diff  \
0                              0.25000   
1                              0.80896   

   significant_strikes_percent_diff_avg_diff  \
0                                   0.145610   
1                                   0.668268   

   clinch_significant_strikes_percent_diff_avg_diff  \
0                                         -1.000000   
1                                         -0.147123   

   distance_body_strikes_percent_diff_avg_diff  \
0                                     0.000000   
1                                     1.712221   

   distance_leg_strikes_percent_diff_avg_diff  \
0                                         0.0   
1                                        -1.0   

   clinch_leg_strikes_percent_diff_avg_diff  \
0                                      -1.0   
1                                      -1.0   

   distance_strikes_percent_diff_avg_diff  \
0                               -1.000000   
1                                0.938695   

   clinch_head_strikes_percent_diff_avg_diff  \
0                                  -1.000000   
1                                   1.222222   

   distance_head_strikes_percent_diff_avg_diff  \
0                                    -1.000000   
1                                     1.945235   

   clinch_body_strikes_percent_diff_avg_diff  
0                                       -1.0  
1                                       -1.0  

[2 rows x 676 columns]

## Convert outcomes to a boolean

In [29]:
df['outcome'] = df.f1_outcome.map(lambda x: 1 if x=='Win' else 0)

## Add reach and height advantage

In [30]:
df['f1_reach_adv'] = df.apply(lambda x: x['f1_reach'] - x['f2_reach'], axis=1)
df['f1_height_adv'] = df.apply(lambda x: x['f1_height'] - x['f2_height'], axis=1)

## Train test split

In [31]:
unique_ids = list(set(df.f1_fighterid))
test_full = df[0:0]
train_full = df[0:0]


for fighter_id in unique_ids:
    sub_df = df[df.f1_fighterid == fighter_id].sort_values('date', ascending=False)
#     print(sub_df.shape[0])
#     print(fighter_id)
    if sub_df.shape[0] > 6:
        test_records = sub_df[:2]
        train_records = sub_df[2:]
        test_full = pd.concat([test_full, test_records], axis=0)
        train_full = pd.concat([train_full, train_records], axis=0)
    elif sub_df.shape[0] > 3:
        test_records = sub_df[:1]
        train_records = sub_df[1:]
        test_full = pd.concat([test_full, test_records], axis=0)
        train_full = pd.concat([train_full, train_records], axis=0)
    else:
        train_records = sub_df
        train_full = pd.concat([train_full, train_records], axis=0)

# Split out X and Y

In [32]:
label = 'outcome'
features = [col for col in train_full.columns if col != label]
y_train = train_full[label]
X_train = train_full[features]

# Drop non-number features
X_train = X_train.select_dtypes(exclude='object')
# Drop non-feature columns
X_train = X_train.drop(columns=['eventid','fightid','f1_fighterid','f2_fighterid'])

In [33]:
label = 'outcome'
features = [col for col in test_full.columns if col != label]
y_test = test_full[label]
X_test = test_full[features]

# Drop non-number features
X_test = X_test.select_dtypes(exclude='object')
# Drop non-feature columns
X_test = X_test.drop(columns=['eventid','fightid','f1_fighterid','f2_fighterid'])

In [34]:
X_train.shape, X_test.shape

((2704, 668), (510, 668))

## Export the Training and Test Data

In [37]:
with open('../../02_Data/02_Processed_Data/X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)

with open('../../02_Data/02_Processed_Data/y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)

with open('../../02_Data/02_Processed_Data/X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)

with open('../../02_Data/02_Processed_Data/y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)

## Export Train_2

In [38]:
df.to_csv('../../02_Data/02_Processed_Data/train_2.csv')